In [2]:
from utils.imports import *

In [3]:
from utils.start_spark import spark
! start http://localhost:4040/jobs/

In [5]:
%%capture 
from files.dfs import *
readAllCsvs(spark, infer = False)

# Cleanup

Ao fazer uma pesquisa adicional sobre esta base de dados pudemos observar [várias inconsistências nela](https://www.kaggle.com/code/abaojiang/eda-on-game-progress), como número de checkpoints não adequados, saltos de `index`es fora de checkpoints, e *primary keys* duplciadas.

In [ ]:
# TODO

# Transformação

In [27]:
splited = train_labels.df \
    .select(
        split("session_id", "_").alias("both"),
        "correct"
    ).select(
        col("both")[0].alias("session_id"),
        col("both")[1].alias("question"),
        col("correct").alias("isCorrect")
    ).cache()
splited.groupby("session_id").count().sort(col("count").asc()).limit(3)

session_id,count
20100209442654932,18
20100216323445964,18
20100308255503228,18


In [32]:
(pivoted := splited.groupby("session_id").pivot("question").agg(first("isCorrect"))).cache()
pivoted.limit(5)

session_id,q1,q10,q11,q12,q13,q14,q15,q16,q17,q18,q2,q3,q4,q5,q6,q7,q8,q9
20110512573169570,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
20100216323445964,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1
21020308080777748,1,0,1,1,0,0,0,1,1,1,1,1,0,1,1,0,1,0
21040512382351228,1,0,1,1,0,1,0,0,1,1,0,1,0,0,1,1,0,0
22030108542877704,0,1,0,1,0,0,0,0,0,1,1,1,0,0,0,1,1,0


In [40]:
sessions = train.df.groupby('session_id') \
    .agg(
        max(col("index")).alias("max_index"),
        
    ).withColumn(
    "id",
    row_number().over(Window.orderBy(monotonically_increasing_id()))-1
)
sessions.limit(10)

session_id,max(index)
20100114570801148,1421
20100308381327090,1089
20100016070249504,941
20100109380136844,1796
20100112570372764,1535
20100113554333536,1143
20100114561049920,812
20100216144092944,1021
20090312431273200,931
20100114164175070,892


In [38]:
# (train_joined := sessions.df.join(pivoted, train.df.session_id == pivoted.session_id)).cache()
# train_joined.limit(10)
# wrongs

session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,session_level,session_id,q1,q10,q11,q12,q13,q14,q15,q16,q17,q18,q2,q3,q4,q5,q6,q7,q8,q9
20090312431273200,0,0,cutscene_click,basic,0,null,-413.99140522263224,-159.31468617022784,380.0,494.0,null,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,1,1323,person_click,basic,0,null,-413.99140522263224,-159.31468617022784,380.0,494.0,null,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,2,831,person_click,basic,0,null,-413.99140522263224,-159.31468617022784,380.0,494.0,null,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,3,1147,person_click,basic,0,null,-413.99140522263224,-159.31468617022784,380.0,494.0,null,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,4,1863,person_click,basic,0,null,-412.99140522263224,-159.31468617022784,381.0,494.0,null,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,5,3423,person_click,basic,0,null,-412.99140522263224,-157.31468617022784,381.0,492.0,null,"Sure thing, Jo. Grab your notebook and come ups...",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,6,5197,person_click,basic,0,null,478.48507948953636,-199.9716787422361,593.0,485.0,null,"See you later, Teddy.",teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,7,6180,person_click,basic,0,null,503.35512792584535,-168.619912992023,609.0,453.0,null,I get to go to Gramps's meeting!,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,8,7014,person_click,basic,0,null,510.7334420194441,-157.72064191237214,615.0,442.0,null,Now where did I put my notebook?,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1
20090312431273200,9,7946,person_click,basic,0,null,512.048004999717,-153.74363056897224,616.0,438.0,null,\u00f0\u0178\u02dc\u00b4,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,null,null,20090312431273200,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1


Uma forma de transformar a target é, em vês da planeada "dataframe de sessões", teriamos uma "dataframe de perguntas", onde em vês de estar uma dataframe com cada linah uma sessão, estar 18 das mesmas linhas, com a diferença nas questões. Ou seja, a nossa chave primária seria o conjunto da das colunas `session_id` e `question` (categórica ou inteira), em vês de apenas ter como chave primária `session_id`.

| session_id | q1 | q2 | ... | q18 | feature1 | feature2 | ... |
|------------|----|----|-----|-----|----------|----------|-----|
| 1          | 0  | 1  | ... | 1   | 423      | 0        | ... |
| 2          | 1  | 0  | ... | 1   | 231      | 1        | ... |
| 3          | 1  | 1  | ... | 1   | 345      | 1        | ... |

para

| session_id | question | answer | feature1 | feature2 | ... |
|------------|----------|--------|----------|----------|-----|
| 1          | 1        | 0      | 423      | 0        | ... |
| 1          | 2        | 1      | 423      | 0        | ... |
| ...        | ...      | ...    | ...      | ...      | ... |
| 1          | 18       | 1      | 423      | 0        | ... |
| 2          | 1        | 1      | 231      | 1        |     |
| ...        | ...      | ...    | ...      | ...      | ... |

Esta opção iria facilitar o processo de previsão, pois podemos usar métodos simples como regressão logística, usando `question` como uma feature também.

Uma segunda opção seria usar classificação *multi-label*, sendo que temos vários `targets`, mas estes só têm uma carnidalidade de 2. Isto iria aumentar a complexidade do projeto (em termos de pesquisa). Estamos também limitados pela utilização de MLLIB do spark.

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.ml.classification.OneVsRest.html#pyspark.ml.classification.OneVsRest

A terceira opção seria treinar 18 modelos, e criar um algoritmo para escolher um modelos na precisão. Essencialmente, este é o método de `binary relevance`, pertencendo à segunda opção.

Nota: da forma como os dados estão agora formatados, essencialmente é o mesmo problema se tivéssemos em vês de uma matriz de 